In [1]:
import hopsworks
import pandas as pd
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os
from sklearn.ensemble import RandomForestRegressor

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


/home/reza/dev/repos/flight-delay-predictor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/197786
Connected. Call `.close()` to terminate connection gracefully.


#### Get or Create a new feature view

In [3]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
flight_fg = fs.get_feature_group(name="flight_data_v3", version=1)
query = flight_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="flight_data_v3",
                                  version=1,
                                  description="Read from Flight Delay dataset",
                                  labels=["dep_delay_new"],
                                  query=query)

#### Create a new train test split

In [4]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (102.18s) 


#### Read from existing train test split

In [5]:
#flight_fg = fs.get_feature_group(name="flight_data", version=1)
#query = flight_fg.select_all()
feature_view = fs.get_feature_view(name="flight_data_v3",version=1)
# get a training dataset with train and test splits
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(training_dataset_version=1)

Finished: Reading data from Hopsworks, using ArrowFlight (34.50s) 


In [6]:
X_train

,month,day_of_month,day_of_week,origin_airport_id,origin_wac,dest_airport_id,dest_wac,crs_dep_time,crs_arr_time,hourlydewpointtemperature,hourlydrybulbtemperature,hourlyprecipitation,hourlyrelativehumidity,hourlystationpressure,hourlyvisibility,hourlywinddirection,hourlywindgustspeed,hourlywindspeed
5,5,19,4,16,91,10,91,600,733,8.9,14.4,0.00,70.0,1015.58,16.09,300,0.0,7.0
6,5,13,5,4,21,0,34,1444,1707,16.7,20.0,0.07,81.0,1022.01,16.09,160,0.0,3.0
8,6,14,1,7,74,9,85,1529,1646,21.7,36.1,0.00,43.0,1008.47,16.09,70,0.0,4.0
12,10,20,3,13,41,14,23,1141,1435,10.6,20.0,0.00,55.0,991.20,16.09,180,0.0,4.0
13,4,5,2,16,91,15,93,1645,1902,5.6,16.7,0.00,48.0,1016.93,16.00,280,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597375,8,28,6,13,41,9,85,900,1053,22.8,28.9,0.00,70.0,993.91,16.09,220,0.0,4.0
1597377,6,11,6,0,34,7,74,1724,1838,17.8,30.6,0.00,46.0,974.26,16.09,210,0.0,2.0
1597379,12,3,6,12,33,0,34,800,1001,17.8,26.7,0.00,58.0,1024.38,16.09,70,10.0,7.0
1597380,11,20,7,11,33,10,91,700,932,7.8,15.0,0.00,62.0,1019.64,16.00,10,0.0,6.0


In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1277905 entries, 5 to 1597381
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   month                      1277905 non-null  int64  
 1   day_of_month               1277905 non-null  int64  
 2   day_of_week                1277905 non-null  int64  
 3   origin_airport_id          1277905 non-null  int64  
 4   origin_wac                 1277905 non-null  int64  
 5   dest_airport_id            1277905 non-null  int64  
 6   dest_wac                   1277905 non-null  int64  
 7   crs_dep_time               1277905 non-null  int64  
 8   crs_arr_time               1277905 non-null  int64  
 9   hourlydewpointtemperature  1277905 non-null  float64
 10  hourlydrybulbtemperature   1277905 non-null  float64
 11  hourlyprecipitation        1277905 non-null  float64
 12  hourlyrelativehumidity     1277905 non-null  float64
 13  hourlystationpres

In [8]:
y_train

,dep_delay_new
5,0.0
6,3.0
8,0.0
12,0.0
13,15.0
...,...
1597375,0.0
1597377,0.0
1597379,0.0
1597380,0.0


In [9]:
y_train.values.ravel()

array([0., 3., 0., ..., 0., 0., 0.])

#### Random Forest Regressor Model

In [10]:
# Initialize the Random Forest classifier
model_RF = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF.fit(X_train, y_train)
# Predict labels for the test set
y_pred = model_RF.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

RMSE: 48.709187931086795
MAE: 20.532364744325477
R-squared: 0.04508392314535847
Mean Squared Error (MSE): 2372.5849889059314


In [11]:
y_train_np=y_train.values.ravel()
X_train_np=X_train.values
X_test_np = X_test.values
y_test_np=y_test.values.ravel()

In [12]:
# Initialize the Random Forest classifier
model_RF_np = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF_np.fit(X_train_np, y_train_np)
# Predict labels for the test set
y_pred_np = model_RF_np.predict(X_test_np)

mse = mean_squared_error(y_test_np, y_pred_np)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test_np, y_pred_np)

# Calculate R-squared
r2 = r2_score(y_test_np, y_pred_np)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

RMSE: 48.709187931086795
MAE: 20.532364744325477
R-squared: 0.04508392314535847
Mean Squared Error (MSE): 2372.5849889059314


#### Linear Regression

In [13]:
# Initialize the Linear Regression model
model_lin_reg = LinearRegression()

# Fit the model on the training data
model_lin_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_lin_reg.predict(X_test)

# Calculate the Mean Squared Error (MSE) on test set

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

RMSE: 49.3797988511361
MAE: 21.26723084662274
R-squared: 0.01860902509753326
Mean Squared Error (MSE): 2438.364534578662


#### Neural Network

In [14]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [15]:
# Normalize features (optional but often recommended for neural networks)
# outputs np arrays
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_test_scaled = scaler.transform(X_test_tensor)

In [16]:
# Convert the scaled NumPy arrays back to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

In [17]:
# Convert to PyTorch Dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [19]:
# Initialize the model
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    #convert to rmse
    train_loss = running_loss / len(train_loader)
    train_rmse = torch.sqrt(torch.tensor(train_loss))
    
    # Calculate validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
        val_loss /= len(val_loader)

    val_rmse = torch.sqrt(torch.tensor(val_loss))

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss MSE: {train_loss}, Train Loss RMSE: {train_rmse}, Val Loss MSE: {val_loss}, Val RMSE: {val_rmse}")

Epoch 1/25, Train Loss MSE: 2436.0096353200615, Train Loss RMSE: 49.3559455871582, Val Loss MSE: 2401.695124341127, Val RMSE: 49.0070915222168
Epoch 2/25, Train Loss MSE: 2420.243494720478, Train Loss RMSE: 49.19596862792969, Val Loss MSE: 2390.8402726405707, Val RMSE: 48.89622116088867
Epoch 3/25, Train Loss MSE: 2413.670301604495, Train Loss RMSE: 49.129119873046875, Val Loss MSE: 2391.7753815735, Val RMSE: 48.90578079223633
Epoch 4/25, Train Loss MSE: 2409.703164114044, Train Loss RMSE: 49.08872604370117, Val Loss MSE: 2388.125718478973, Val RMSE: 48.86845397949219
Epoch 5/25, Train Loss MSE: 2405.956850065492, Train Loss RMSE: 49.05055236816406, Val Loss MSE: 2381.561928323446, Val RMSE: 48.80125045776367
Epoch 6/25, Train Loss MSE: 2403.989859775454, Train Loss RMSE: 49.03049850463867, Val Loss MSE: 2378.9665459432663, Val RMSE: 48.77465057373047
Epoch 7/25, Train Loss MSE: 2402.566209946548, Train Loss RMSE: 49.0159797668457, Val Loss MSE: 2376.9631312848664, Val RMSE: 48.7541084

#### Save to Hopsworks Model Registry 

In [25]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="flight delay model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/flight_delay_model.pkl")
#fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
#input_schema = Schema(X_train_tensor)
#output_schema = Schema(y_train_tensor)
#model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
fligt_delay_model = mr.python.create_model(
    name="flight_delay_model", 
    #metrics={"accuracy" : metrics['accuracy']},
    metrics={"val RMSE" : val_rmse.item()},
    #model_schema=model_schema,
    description="Flight Delay Predictor no schema"
)

# Upload the model to the model registry, including all files in 'model_dir'
fligt_delay_model.save(model_dir + "/flight_delay_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 46277/46277 elapsed<00:01 remaining<00:001,  2.92it/s]
Model export complete: 100%|██████████| 6/6 [00:08<00:00,  1.34s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/197786/models/flight_delay_model/2


Model(name: 'flight_delay_model', version: 2)

### Example showing how to load the NN model from Model Registry and how to use it for inference

Class definition needed due to how python loads saved pytorch models. The class definition is used to load the pickle model. If the pytorch model was saved as a state dict it could also be loaded using state_dict without defining class definition

In [28]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

mr = project.get_model_registry()
model = mr.get_model("flight_delay_model", version=2)
model_dir = model.download()
model = joblib.load(model_dir + "/flight_delay_model.pkl")


Connected. Call `.close()` to terminate connection gracefully.


In [29]:
#Data sample used for inference
i_test=0
a=X_train.iloc[i_test]
a=torch.tensor(a, dtype=torch.float32)

#Fit scaler to how it was fitted during training
scaler = StandardScaler()
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)

#transform data sample
a=scaler.transform(a.reshape(1, -1))
print(a)
print(X_train_tensor[i_test])

# transform np array to torch tensor
a_tensor=torch.tensor(a, dtype=torch.float32)

print(a_tensor)
print(model(a_tensor))
print(y_train.iloc[i_test])

[[-0.52922551  0.36525144  0.0032831   1.5751317   1.34140484  0.43369134
   1.35978058 -1.44706085 -1.37389841 -0.0266942  -0.40259642 -0.10026567
   0.42778787  0.5549998   0.31664326  1.13461926 -0.35559528  1.17732389]]
tensor([-0.5292,  0.3653,  0.0033,  1.5751,  1.3414,  0.4337,  1.3598, -1.4471,
        -1.3739, -0.0267, -0.4026, -0.1003,  0.4278,  0.5550,  0.3166,  1.1346,
        -0.3556,  1.1773])
tensor([[-0.5292,  0.3653,  0.0033,  1.5751,  1.3414,  0.4337,  1.3598, -1.4471,
         -1.3739, -0.0267, -0.4026, -0.1003,  0.4278,  0.5550,  0.3166,  1.1346,
         -0.3556,  1.1773]])
tensor([[3.6024]], grad_fn=<AddmmBackward0>)
dep_delay_new    0.0
Name: 5, dtype: float64
